# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [70]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [71]:
#set file path and read in csv to dataframe
path = "../output_data/cities.csv"
cities_df = pd.read_csv(path)

cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cidreira,-30.1811,-50.2056,73.04,68,0,9.42,BR,1635364075
1,Vila Franca do Campo,37.7167,-25.4333,66.61,90,100,9.06,PT,1635364075
2,Beringovskiy,63.0500,179.3167,6.96,86,92,16.42,RU,1635364075
3,Qaanaaq,77.4840,-69.3632,20.75,94,100,11.21,GL,1635363977
4,Ponta do Sol,32.6667,-17.1000,70.43,61,7,3.18,PT,1635364075


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [72]:
#check data types for Lat, Lng, Humidity
cities_df.dtypes

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

In [73]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [74]:
#set up variables for locations to plot and weight of points
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"]

In [75]:
#set up gmaps
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

#set up heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)
        
# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [79]:
# Ideal weather conditions:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions.

hotel_df = cities_df.loc[(cities_df["Max Temp"] <= 80) & (cities_df["Max Temp"] >= 70) & (cities_df["Wind Speed"]<= 10) & (cities_df["Cloudiness"] == 0)].copy()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Cidreira,-30.1811,-50.2056,73.04,68,0,9.42,BR,1635364075
27,Chui,-33.6971,-53.4616,77.47,57,0,6.93,UY,1635364016
85,Salalah,17.0151,54.0924,77.09,65,0,2.30,OM,1635363927
97,Kiryat Gat,31.6100,34.7642,70.81,63,0,6.22,IL,1635364096
158,Dhadar,28.3305,75.0632,72.34,40,0,2.73,IN,1635364108
206,Necochea,-38.5473,-58.7368,79.90,54,0,7.43,AR,1635364118
208,San Quintín,30.4833,-115.9500,72.45,45,0,7.14,MX,1635364119
275,Gwadar,25.1216,62.3254,78.39,64,0,4.47,PK,1635364133
401,Antsohihy,-14.8796,47.9875,76.05,80,0,2.28,MG,1635364160
449,Saint-Pierre,-21.3393,55.4781,71.28,71,0,6.91,RE,1635364156


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [80]:
#empty variables to store hotel data in
hotel_lat = []
hotel_lng = []
hotel_name = []

#use google places api
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up a parameters dictionary for api query
params = {
    "keyword": "hotel",
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

for row in best_weather.itertuples():     
    #set location in query url to each set of lat/lng
    #************************change row locations once df output fixed in weatherpy*****************
    params['location'] = str(row[2]) + ", " + str(row[3])      
    
    #call API for data
    response = requests.get(base_url, params=params)
    nearby_hotel_data = response.json()
    
    #append api data to list variable
    hotel_lat.append(nearby_hotel_data["results"][0]["geometry"]['location']['lat'])
    hotel_lng.append(nearby_hotel_data["results"][0]["geometry"]['location']['lng'])
    hotel_name.append(nearby_hotel_data["results"][0]["name"])

IndexError: list index out of range

In [67]:
#save gathered hotel info to dataframe
hotel_df["Hotel Name"] = hotel_name
hotel_df["Hotel Lat"] = hotel_lat
hotel_df["Hotel Lng"] = hotel_lng

In [68]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Hotel Lat", "Hotel Lng"]]

In [69]:
#set up gmaps figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

#save hotel locations and info to variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

#add layers
fig.add_layer(markers)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))